**這個記事本只能在Google Colab中使用**  
這個記事本為技術型態識別之物件偵測模型之建置程序，程序是參照「[TensorFlow 2 Object Detection API With Google Colab](https://medium.com/swlh/tensorflow-2-object-detection-api-with-google-colab-b2af171e81cc)」的步驟；並由「[第二十七天：簡述物件偵測技術.ipynb](https://github.com/yilintung/IronMan2023/blob/main/%E7%AC%AC%E4%BA%8C%E5%8D%81%E4%B8%83%E5%A4%A9%EF%BC%9A%E7%B0%A1%E8%BF%B0%E7%89%A9%E4%BB%B6%E5%81%B5%E6%B8%AC%E6%8A%80%E8%A1%93.ipynb)」進行修改。  

In [ ]:
#步驟4 ： 掛載Google雲端硬碟

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#步驟5 ： 下載TensorFlow Model Garden

%cd '/content/gdrive/My Drive/TensorFlow'

!git clone https://github.com/tensorflow/models.git

In [ ]:
#步驟6 ： 安裝一些必要工具

!apt-get install protobuf-compiler
!pip install lxml
!pip install Pillow

!pip install Cython pandas tf-slim lvis

In [ ]:
#步驟7 ： 編譯Protobuf程式庫

#cd into 'TensorFlow/models/research'
%cd '/content/gdrive/My Drive/TensorFlow/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
#步驟8 ： 設定環境

import os
import sys
os.environ['PYTHONPATH']+=":/content/gdrive/My Drive/TensorFlow/models"
sys.path.append("/content/gdrive/My Drive/TensorFlow/models/research")

In [ ]:
##步驟9 ： 構建並安裝setup.py

!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
#步驟10 ： 測試安裝

#cd into 'TensorFlow/models/research/object_detection/builders/'
%cd '/content/gdrive/My Drive/TensorFlow/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

In [ ]:
#步驟11 ： 產生TFrecords

%cd '/content/gdrive/My Drive/TensorFlow/scripts/preprocessing'

!python generate_tfrecord.py -x '/content/gdrive/My Drive/TensorFlow/workspace/stock_pattern_recorginition/images/train' -l '/content/gdrive/My Drive/TensorFlow/workspace/stock_pattern_recorginition/annotations/label_map.pbtxt' -o '/content/gdrive/My Drive/TensorFlow/workspace/stock_pattern_recorginition/annotations/train.record'
!python generate_tfrecord.py -x '/content/gdrive/My Drive/TensorFlow/workspace/stock_pattern_recorginition/images/test' -l '/content/gdrive/My Drive/TensorFlow/workspace/stock_pattern_recorginition/annotations/label_map.pbtxt' -o '/content/gdrive/My Drive/TensorFlow/workspace/stock_pattern_recorginition/annotations/test.record'

In [ ]:
#步驟14 ： 啟動TensorBoard

%cd '/content/gdrive/My Drive/TensorFlow/workspace/stock_pattern_recorginition'

%load_ext tensorboard
%tensorboard --logdir=models/my_ssd_mobilenet_v2

In [ ]:
#步驟15 ： 訓練模型

!python model_main_tf2.py --model_dir=models/my_ssd_mobilenet_v2 --pipeline_config_path=models/my_ssd_mobilenet_v2/pipeline.config

In [ ]:
#步驟16 - 導出經過訓練的模型

!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_mobilenet_v2/pipeline.config --trained_checkpoint_dir ./models/my_ssd_mobilenet_v2/ --output_directory ./exported-models/my_model

In [ ]:
#步驟17 - 測試模型

import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL="/content/gdrive/My Drive/TensorFlow/workspace/stock_pattern_recorginition/exported-models/my_model/saved_model"

print('Loading model...', end='')

detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)

print('Done!')

In [ ]:
#步驟18 - 測試模型

category_index=label_map_util.create_category_index_from_labelmap("/content/gdrive/My Drive/TensorFlow/workspace/stock_pattern_recorginition/annotations/label_map.pbtxt",use_display_name=True)

In [ ]:
#步驟19 - 測試模型

img=['/content/img1.jpg','/content/img2.jpg']
print(img)

In [ ]:
#步驟20 - 執行推論

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

for image_path in img:

    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()
    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor=tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor=input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections=detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy()
                   for key,value in detections.items()}
    detections['num_detections']=num_detections

    # detection_classes should be ints.
    detections['detection_classes']=detections['detection_classes'].astype(np.int64)

    image_np_with_detections=image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=1,     #max number of bounding boxes in the image
          min_score_thresh=.3,      #min prediction threshold
          agnostic_mode=False)
    %matplotlib inline
    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
    plt.show()